# Regularity analysis: interactive notebook

The code below can be edited and run interactively, and creates a graphical interface for exploring a mathematical neuroscience model. To try it out, click the "Cell" menu, then "Run all", and scroll down to the bottom. You can see the effect of changing the parameters $\tau$ and ``refractory``.

This is being computed and run as you change the parameters, so there is a few seconds delay in showing the results after changing the parameters. By default it runs in "fast" mode so the quality is low. If you want to look in more detail at a particular set of parameters, use the drop down list to select a higher quality (but it will take longer to compute it).

Specifically, it is showing the coefficient of variation of the interspike interval distribution for a noise-driven leaky integrate and fire neuron following the stochastic differential equation:

$$\tau\frac{\mathrm{d}v}{\mathrm{d}t}=\mu-v+\sigma\,\xi\,\tau^{-1/2}$$

Spikes are emitted when $v\ge 1$ and $v$ is then reset to 0 and held for ``refractory``.

The solid lines show contours for the CV, and the dashed lines show contours for the firing rate of this neuron.

In [ ]:
%matplotlib inline
from brian2 import *
from brian2tools import *
from scipy.ndimage.interpolation import zoom
from scipy.ndimage.filters import gaussian_filter
from ipywidgets import interact, interactive, fixed
import ipywidgets
from IPython.display import display
from matplotlib import cm
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore")
prefs.codegen.target = 'numpy'

In [ ]:
def nan_gaussian_filter(x, sigma, num_passes):
    z = full_like(x, nan)
    for cursigma in linspace(sigma, 0, num_passes+1)[:-1]:
        y = gaussian_filter(x, cursigma, mode='nearest')
        z[isnan(z)] = y[isnan(z)]
    return z

In [ ]:
def map_mu_sigma(tau_ms, t_ref_ms,
                 #duration_ms=150, skip_ms=50, N=20, blur_width=0.1,
                 quality=1,
                ):
    tau = tau_ms*ms
    t_ref = t_ref_ms*ms
    if quality==0:
        N = 15
        duration = 50*ms
        skip = 25*ms
        blur_width = 0.05
    elif quality==1:
        N = 20
        duration = 100*ms
        skip = 50*ms
        blur_width = 0.1
    elif quality==2:
        N = 30
        duration = 200*ms
        skip = 50*ms
        blur_width = 0.1
    elif quality==3:
        N = 50
        duration = 1000*ms
        skip = 100*ms
        blur_width = 0.05
    mu_range = linspace(0.5, 4, N)
    s_range = linspace(0, 1.2, N)
    all_s, all_mu = meshgrid(s_range, mu_range)
    all_sigma = all_s*all_mu
    eqs = '''
    dv/dt = (mu-v)/tau+sigma*tau**-0.5*xi : 1 (unless refractory)
    mu : 1
    sigma : 1
    '''
    G = NeuronGroup(N*N, eqs, threshold='v>1', reset='v=0', refractory=t_ref,
                    method='euler')
    G.mu = all_mu.flatten()
    G.sigma = all_sigma.flatten()
    run(skip)
    M = SpikeMonitor(G)
    run(duration-skip)
    rate = array(reshape(M.count[:]/(duration-skip), (N, N)))
    trains = M.spike_trains()
    cvs = zeros(N*N)
    for i in xrange(N*N):
        isi = diff(sorted(trains[i]))
        cvs[i] = std(isi)/mean(isi)
    cvs.shape = rate.shape
    if blur_width:
        rate = nan_gaussian_filter(rate, blur_width*N, 10)
        cvs = nan_gaussian_filter(cvs, blur_width*N, 10)
    rate = zoom(rate, 100./N, order=1)
    cvs = zoom(cvs, 100./N, order=1)
    imshow(cvs, origin='lower left', interpolation='nearest', aspect='auto',
           extent=(amin(s_range), amax(s_range), amin(mu_range), amax(mu_range)),
           cmap=cm.YlGnBu_r, vmin=0, vmax=1)
    ylabel(r'$\mu$')
    xlabel(r'$\sigma/\mu$')
    cb = colorbar()
    cb.set_label('CV', rotation=270, labelpad=20)    
    cs_cv = contour(cvs, origin='lower', aspect='auto',
                    levels=[0.35, 0.8], #colors=['w']*len(cv_levels),
                    colors='k',
                    extent=(amin(s_range), amax(s_range), amin(mu_range), amax(mu_range)),
                    )
    clabel(cs_cv, colors='k', inline=True, fmt='%.2f')
    cs_rate = contour(rate, origin='lower', aspect='auto', linestyles='dashed',
                      levels=[50, 400], colors='k',
                      extent=(amin(s_range), amax(s_range), amin(mu_range), amax(mu_range)),
                      )
    clabel(cs_rate, colors='k', inline=True, fmt='%d sp/s')

In [ ]:
interact(map_mu_sigma,
         tau_ms=ipywidgets.FloatSlider(value=5, min=0.1, max=20, step=0.1, continuous_update=False,
                                       description=r"$\tau$ (ms)"),
         t_ref_ms=ipywidgets.FloatSlider(value=0.1, min=0, max=5, step=0.1, continuous_update=False,
                                         description=r"refractory (ms)"),
         quality=ipywidgets.Dropdown(options=OrderedDict([('Very fast', 0),
                                                          ('Fast', 1),
                                                          ('Medium', 2),
                                                          ('High / slow', 3)]),
                                     value=1, description="Quality"),
        );